# Тестирование TreeTagger

Если TreeTagger не установлен, прогоните следующую ячейку (скрипт установки в папке с заданием):

In [ ]:
!sh get_treetagger.sh

In [2]:
!echo 'Я хочу съесть яблоко!' | cmd/tree-tagger-russian 

	reading parameters ...
	tagging ...
Я	P-1-snn	я
хочу	Vmip1s-a-e	хотеть
съесть	Vmn----a-p	съесть
яблоко	Ncnsan	яблоко
!	SENT	!
	 finished.


Копипаста из семинара: открываю файл и обрабатываю тексты по инструкции:

In [ ]:
from lxml import etree
from collections import defaultdict

open_corpora = etree.fromstring(open('annot.opcorpora.no_ambig_strict.xml', 'rb').read())

corpus = open('corpus_train.txt', 'w')
vocab = defaultdict(set)
tags = set()

for sentence in open_corpora.xpath('//tokens'):
    length = len(sentence.xpath('token'))
    ended = False
    for i,token in enumerate(sentence.xpath('token')):
        word = token.xpath('@text')
        gram_info = token.xpath('tfr/v/l/g/@v')
        
        if (i+1)==length and gram_info[0] == 'PNCT':
            gram_info = ['SENT']
            ended = True
        
            
        corpus.write(word[0] + '\t' + ','.join(gram_info) + '\n')
        lemma = token.xpath('tfr/v/l/@t')[0]
        vocab[word[0].lower()].add((','.join(gram_info), lemma.lower()))
        tags.add(','.join(gram_info))
    
    if not ended:
        corpus.write('.\tSENT\n')

In [ ]:
f = open('lexicon.txt', 'w')

for word in vocab:
    f.write(word + '\t')
    f.write('\t'.join([' '.join(pair) for pair in vocab[word]]))
    f.write('\n')
# f.write('SENT\tSENT .')
f.close()

In [ ]:
f = open('open_class.txt', 'w')

f.write('\n'.join([tag for tag in tags if 'NOUN' in tag or 'VERB' in tag or 'ADJF' in tag]))
f.close()